In [175]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import calendar 

In [217]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate, KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [177]:
#importing csvs
features = pd.read_csv("features+data.csv")
sales = pd.read_csv("sales+data.csv")
stores = pd.read_csv("stores+data.csv")
display(features.head(),sales.head(),stores.head())

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [178]:
#analysing feature data frame
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [179]:
features.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [180]:
features.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000


In [181]:
features['IsHoliday'].value_counts()

False    7605
True      585
Name: IsHoliday, dtype: int64

In [182]:
features['Date'] = pd.to_datetime(features['Date'])
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         8190 non-null   int64         
 1   Date          8190 non-null   datetime64[ns]
 2   Temperature   8190 non-null   float64       
 3   Fuel_Price    8190 non-null   float64       
 4   MarkDown1     4032 non-null   float64       
 5   MarkDown2     2921 non-null   float64       
 6   MarkDown3     3613 non-null   float64       
 7   MarkDown4     3464 non-null   float64       
 8   MarkDown5     4050 non-null   float64       
 9   CPI           7605 non-null   float64       
 10  Unemployment  7605 non-null   float64       
 11  IsHoliday     8190 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), int64(1)
memory usage: 712.0 KB


In [183]:
#analysing sales dataframe
print(sales.info())
print("***************")
print("Null values:")
print(sales.isnull().sum())
display(sales.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
None
***************
Null values:
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64


,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


In [184]:
sales['Date'] = pd.to_datetime(sales['Date'])
sales['Year'], sales['Month'], sales['Day'] = sales.Date.dt.year, sales.Date.dt.month, sales.Date.dt.day
print(sales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Year          421570 non-null  int64         
 6   Month         421570 non-null  int64         
 7   Day           421570 non-null  int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(5)
memory usage: 22.9 MB
None


In [185]:
display(sales[['Month', 'Weekly_Sales']].groupby('Month', as_index = False).agg({'Weekly_Sales':'mean'}))
print("*************************")
display(sales[['Day', 'Weekly_Sales']].groupby('Day', as_index = False).agg({'Weekly_Sales':'mean'}))

,Month,Weekly_Sales
0,1,14503.308110
1,2,16026.823127
2,3,15631.676728
3,4,15638.149799
4,5,15850.122787
5,6,16258.141930
6,7,15563.149206
7,8,16012.023938
8,9,15378.844836
9,10,15728.044488


*************************


,Day,Weekly_Sales
0,1,14862.139543
1,2,16204.119757
2,3,15673.992298
3,4,16293.445430
4,5,16006.023215
5,6,16643.308298
6,7,16629.640035
7,8,16154.589662
8,9,15831.878770
9,10,15427.488477


In [186]:
#Analysing stores data frame
print(stores.info())
print("Null values:")
print(stores.isnull().sum())
display(stores.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB
None
Null values:
Store    0
Type     0
Size     0
dtype: int64


,Store,Size
count,45.000000,45.000000
mean,23.000000,130287.600000
std,13.133926,63825.271991
min,1.000000,34875.000000
25%,12.000000,70713.000000
50%,23.000000,126512.000000
75%,34.000000,202307.000000
max,45.000000,219622.000000


In [187]:
sales_store = sales.merge(stores, on = ['Store'], how='left')
print(sales_store.shape)
sales_store.columns

(421570, 10)


Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Year', 'Month',
       'Day', 'Type', 'Size'],
      dtype='object')

In [188]:
merged_data = pd.merge(sales_store, features, how = 'left', on=['Store','Date','IsHoliday'])
print(merged_data.shape)
merged_data.columns

(421570, 19)


Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Year', 'Month',
       'Day', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1',
       'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment'],
      dtype='object')

In [189]:
merged_data.dropna(inplace=True)

In [190]:
merged_data

,Store,Dept,Date,Weekly_Sales,IsHoliday,Year,Month,Day,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
92,1,1,2011-11-11,18689.54,False,2011,11,11,A,151315,59.11,3.297,10382.90,6115.67,215.07,2406.62,6551.42,217.998085,7.866
93,1,1,2011-11-18,19050.66,False,2011,11,18,A,151315,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57,218.220509,7.866
94,1,1,2011-11-25,20911.25,True,2011,11,25,A,151315,60.14,3.236,410.31,98.00,55805.51,8.00,554.92,218.467621,7.866
95,1,1,2011-02-12,25293.49,False,2011,2,12,A,151315,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866
96,1,1,2011-09-12,33305.92,False,2011,9,12,A,151315,43.93,3.158,4640.65,19.00,105.02,3639.42,14461.82,218.961846,7.866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421561,45,98,2012-08-31,346.04,False,2012,8,31,B,118221,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,191.461281,8.684
421562,45,98,2012-07-09,352.44,True,2012,7,9,B,118221,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,191.577676,8.684
421564,45,98,2012-09-21,467.30,False,2012,9,21,B,118221,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,191.856704,8.684
421565,45,98,2012-09-28,508.37,False,2012,9,28,B,118221,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684


In [191]:
highstore = merged_data[['Store', 'Weekly_Sales']].groupby('Store', as_index = False).agg({'Weekly_Sales':'sum'})
highstore.sort_values(by = 'Weekly_Sales',ascending=False)

,Store,Weekly_Sales
19,20,95584470.15
3,4,89317174.91
12,13,88040357.30
1,2,80748967.22
13,14,76185715.59
26,27,72882457.89
5,6,67525366.48
0,1,66294479.23
9,10,63329254.85
38,39,61945882.77


In [192]:
highdept = merged_data[['Dept', 'Weekly_Sales']].groupby('Dept', as_index = False).agg({'Weekly_Sales':'sum'})
highdept.sort_values(by = 'Weekly_Sales',ascending=False)

,Dept,Weekly_Sales
73,92,1.250757e+08
76,95,1.091220e+08
36,38,9.124449e+07
60,72,8.513064e+07
38,40,7.392061e+07
...,...,...
45,47,2.010900e+03
49,51,3.765000e+02
63,78,2.024000e+02
37,39,4.072000e+01


In [193]:
#Create dummy variables
def multipledummies(df, non_numerical_columns):
    for i in non_numerical_columns:
        one_hot = pd.get_dummies(df[i], prefix=i)
        df = df.join(one_hot)
        df.drop(i, axis = 1, inplace=True)
    return df

In [194]:
merged_data = multipledummies(merged_data, ['Type'])
merged_data = multipledummies(merged_data,['IsHoliday'])
print(merged_data.shape)
merged_data.head()

(97056, 22)


,Store,Dept,Date,Weekly_Sales,Year,Month,Day,Size,Temperature,Fuel_Price,...,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type_A,Type_B,Type_C,IsHoliday_False,IsHoliday_True
92,1,1,2011-11-11,18689.54,2011,11,11,151315,59.11,3.297,...,215.07,2406.62,6551.42,217.998085,7.866,1,0,0,1,0
93,1,1,2011-11-18,19050.66,2011,11,18,151315,62.25,3.308,...,51.98,427.39,5988.57,218.220509,7.866,1,0,0,1,0
94,1,1,2011-11-25,20911.25,2011,11,25,151315,60.14,3.236,...,55805.51,8.00,554.92,218.467621,7.866,1,0,0,0,1
95,1,1,2011-02-12,25293.49,2011,2,12,151315,48.91,3.172,...,1398.11,2084.64,20475.32,218.714733,7.866,1,0,0,1,0
96,1,1,2011-09-12,33305.92,2011,9,12,151315,43.93,3.158,...,105.02,3639.42,14461.82,218.961846,7.866,1,0,0,1,0


In [196]:
#Drop irrelevant features if there are any (Ex- Date)
merged_data= merged_data.drop(['Date'], axis = 1) 
merged_data.shape

(97056, 21)

In [199]:
print(merged_data.shape)
index_names = merged_data[ merged_data['Weekly_Sales'] <100].index
merged_data.drop(index_names, inplace = True)
print(merged_data.shape)

(97056, 21)
(93081, 21)


In [219]:
#Segregate the target column
X=merged_data.drop('Weekly_Sales',axis=1)
y=merged_data['Weekly_Sales']

#Standardize the independent data using StandardScaler
scaler=StandardScaler()
scaled_data=scaler.fit_transform(X)
scaled_data=pd.DataFrame(scaled_data)
scaled_data.columns=X.columns
scaled_data.head()

X = scaled_data
#Split the above data into two parts
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(74464, 20) (18617, 20) (74464,) (18617,)
